In [1]:
#Imports
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Network parameters
batch_size = 64
img_height = 224
img_width = 224

In [3]:
#Process data
test_folder = '../images/uk_us/test'
preprocessor = keras.applications.resnet50.preprocess_input
test_data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor)
test_set = test_data_generator.flow_from_directory(test_folder, class_mode='binary',shuffle = True,target_size=(img_height,img_width))

Found 374 images belonging to 2 classes.


In [4]:
#Get pretrained model
resnet_model = tf.keras.applications.ResNet50(input_shape=(img_width, img_height, 3), include_top=False, pooling='avg', weights='imagenet')
resnet_model.trainable = False
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [5]:
inputs = keras.Input(shape=(img_height,img_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = resnet_model(inputs, training=False)
x = keras.layers.Dense(1, name='logits')(x)
outputs = keras.layers.Activation('sigmoid',name='sigmoid_out')(x)
model = keras.Model(inputs, outputs)

In [6]:
#compile and set params
#model = keras.models.Sequential([resnet_model, keras.layers.Dense(1, name='logits'), keras.layers.Activation('sigmoid',name='sigmoid_out')])
opt = keras.optimizers.Adam(learning_rate=0.005)
loss = keras.losses.BinaryCrossentropy()
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='binary_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.MeanAbsoluteError(name='mae')
]

model.compile(loss=loss, optimizer=opt, metrics=METRICS)

In [7]:
model.load_weights('./model_checkpoints/uk_us/seq_location_classifier')

In [8]:
res = model.evaluate(test_set)

12/12 [==============================] - 14s 1s/step - loss: 0.7855 - tp: 77.3077 - fp: 71.1538 - tn: 44.3846 - fn: 27.1538 - binary_accuracy: 0.5561 - precision: 0.5105 - recall: 0.7401 - auc: 0.5625 - accuracy: 0.0000e+00 - mae: 0.4833
